<a href="https://colab.research.google.com/github/utkmittal1/Bert_intent_detection/blob/master/bertsequenceclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd  /content/drive/My Drive/bert_model
!ls

/content/drive/My Drive/bert_model
bert_classifier.csv	    dataset_reader.py  saved_model
data.csv		    model.py	       test.py
dataset_reader_original.py  __pycache__        train.py


In [3]:
!pip install transformers

     |████████████████████████████████| 501kB 8.0MB/s 
     |████████████████████████████████| 3.7MB 66.5MB/s 
     |████████████████████████████████| 1.0MB 57.9MB/s 
     |████████████████████████████████| 870kB 34.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=abf3c42fbf449993aa3596b342c76b13e2ef46dbb83605a84fea4b40c1e147c7
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [4]:
from tqdm import tqdm as tqdm
import model as model_py
import dataset_reader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

train_dataloader, val_dataloader = dataset_reader.DatasetReader(train_dev_split= 0.9).dataset_loader()
train_count = len(train_dataloader)
m = model_py.Model(train_count).to(device)
# optim = torch.optim.Adam(params=m.model.parameters(), lr=1e-3)

optim = torch.optim.Adam([
      {"params": m.model.bert.parameters(), "lr": 5e-6},
      {"params": m.model.classifier.parameters(), "lr": 1e-3}
  ], lr=1e-3)


def run_epoch(model, data_loader, optim, training=True):
    if training:
        model.train()
        torch.set_grad_enabled(True)
    else:
        model.eval()
        torch.set_grad_enabled(False)

    tk0 = tqdm(data_loader, total=len(data_loader))
    running_loss = 0
    counter = 0
    counter_correct = 0
    for i, batch in enumerate(tk0):
        if training:
            optim.zero_grad()

        input_ids = batch[0]['intent_ids']
        label_ids = [batch[0]['label_ids']]
        outputs = model(input_ids, label_ids)
        intent_label_id, loss = outputs[:2]

        if intent_label_id.item() == label_ids[0]:
            counter_correct += 1
        counter += 1
        running_loss = running_loss + loss.item()
        accuracy = counter_correct / counter
        if training:
            loss.backward()
            optim.step()

    return running_loss / counter, accuracy, model

print("initial validation run ---------")
print(run_epoch(m, val_dataloader, optim, training=False)[:2])

val_loss_prev= 1
for i in range(20):
  print("epoch" + str(i) + "-"*20)
  print("training:\n")
  train_loss, train_acc, m = run_epoch(m, train_dataloader, optim, training=True)
  print(train_loss, train_acc)
  print("validation:\n")
  val_loss, val_acc = (run_epoch(m, val_dataloader, optim, training=False)[:2])
  print("val_loss: ",val_loss, "val_acc: ", val_acc)
  if val_loss < val_loss_prev :
    torch.save(m.state_dict(), "saved_model/model_{}.model".format(str(i)))
    print("\n saved model: model_{}.model".format(str(i)))
    val_loss_prev = val_loss





cuda


  0%|          | 0/28 [00:00<?, ?it/s]

initial validation run ---------


  0%|          | 1/243 [00:00<00:29,  8.34it/s]

(5.489665337971279, 0.0)
epoch0--------------------
training:



...

val_loss:  0.1365804672241211 val_acc:  0.9642857142857143
epoch19--------------------
training:



 32%|███▏      | 9/28 [00:00<00:00, 85.31it/s]

0.0017672526983567227 1.0
validation:



100%|██████████| 28/28 [00:00<00:00, 98.74it/s]

val_loss:  0.14868770326886857 val_acc:  0.9642857142857143


In [8]:
ut = "For each month in 2017, I want to know how much our sales grew compared to the same month in 2018"

x, y = dataset_reader.DatasetReader(input_utterance=ut).dataset_loader()

input_ids = [b[0] for b in x][0]['intent_ids']
input_ids = input_ids + [0]*(30-len(input_ids))
print(input_ids)
label_ids = [0]
out = m(input_ids, label_ids)
print(out[0].item())
print(out[1].item())




[101, 2005, 2169, 3204, 1999, 2418, 1010, 1045, 2215, 2000, 2113, 2129, 2172, 2256, 4341, 3473, 4102, 2000, 1996, 2168, 3204, 1999, 2760, 102, 0, 0, 0, 0, 0, 0]
12
20.88443374633789


In [0]:
%mkdir "saved_model"

In [5]:
!ls

bert_classifier.csv	    dataset_reader.py  saved_model
data.csv		    model.py	       test.py
dataset_reader_original.py  __pycache__        train.py
